# Libraries

In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Data

In [26]:
data1 = pd.read_csv('data1.csv')

received = data1['packets_received']
forwarded = data1['packets_forwarded']
login_attempts = data1['unusual_login_attempts']
shared_resources = data1['resources_shared']
cpu = data1['cpu_usage']
memory = data1['memory_usage']
storage = data1['storage_usage']
uptime = data1['device_uptime']
latency = data1['latency']
bandwidth = data1['bandwidth_usage']
transferred = data1['data_transferred']
label1 = data1['label']

data2 = pd.read_csv('data2.csv')

received2 = data2['packets_received']
forwarded2 = data2['packets_forwarded']
login_attempts2 = data2['unusual_login_attempts']
shared_resources2 = data2['resources_shared']
cpu2 = data2['cpu_usage']
memory2 = data2['memory_usage']
storage2 = data2['storage_usage']
uptime2 = data2['device_uptime']
latency2 = data2['latency']
bandwidth2 = data2['bandwidth_usage']
transferred2 = data2['data_transferred']
label2 = data2['label']

data3 = pd.read_csv('data3.csv')

received3 = data3['packets_received']
forwarded3 = data3['packets_forwarded']
login_attempts3 = data3['unusual_login_attempts']
shared_resources3 = data3['resources_shared']
cpu3 = data3['cpu_usage']
memory3 = data3['memory_usage']
storage3 = data3['storage_usage']
uptime3 = data3['device_uptime']
latency3 = data3['latency']
bandwidth3 = data3['bandwidth_usage']
transferred3 = data3['data_transferred']
label3 = data3['label']


x1 = np.concatenate([received, received2, received3])
x2 = np.concatenate([forwarded, forwarded2, forwarded3])
x3 = np.concatenate([login_attempts, login_attempts2, login_attempts3])
x4 = np.concatenate([shared_resources, shared_resources2, shared_resources3])
x5 = np.concatenate([cpu, cpu2, cpu3])
x6 = np.concatenate([memory, memory2, memory3])
x7 = np.concatenate([storage, storage2, storage3])
x8 = np.concatenate([uptime, uptime2, uptime3])
x9 = np.concatenate([latency, latency2, latency3])
x10 = np.concatenate([bandwidth, bandwidth2, bandwidth3])
x11 = np.concatenate([transferred, transferred2, transferred3])
y = np.concatenate([label1, label2, label3])




# Normalize and Vectorization

In [27]:
x = [x8, x9, x10, x1, x2, x11, x4, x5, x6, x7, x3]
x = np.hstack([arr.reshape(-1, 1) for arr in x])

print(x.shape)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X_train.shape)
print(X_test.shape)

(2142, 11)
(1713, 11)
(429, 11)


# Model

In [28]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(11,)),
    tf.keras.layers.Dense(8, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 8)              │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 137 (548.00 B)

 Trainable params: 137 (548.00 B)

 Non-trainable params: 0 (0.00 B)

# Weights

In [29]:
# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

# Training

In [22]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train, y_train, 
    epochs=100, 
    batch_size=32, 
    validation_split=0.2, 
    class_weight=class_weights_dict, 
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 7s 82ms/step - accuracy: 0.6012 - loss: 0.8288 - val_accuracy: 0.5598 - val_loss: 0.7974
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.4849 - loss: 0.8049 - val_accuracy: 0.5277 - val_loss: 0.7987
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4418 - loss: 0.8479 - val_accuracy: 0.4927 - val_loss: 0.7979
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4552 - loss: 0.8095 - val_accuracy: 0.4752 - val_loss: 0.7937
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4318 - loss: 0.8149 - val_accuracy: 0.4840 - val_loss: 0.7862
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4042 - loss: 0.8340 - val_accuracy: 0.4227 - val_loss: 0.7891
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4151 - loss: 0.7690 - val_accuracy: 0.4198 - val_loss: 0.7812
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4039 - loss: 0.7929 - val_accuracy: 0.4490 - 

# Testing

In [30]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Predictions
y_pred = (model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred))

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7120 - loss: 0.7179
Test Loss: 0.7359, Test Accuracy: 0.7016
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
              precision    recall  f1-score   support

           0       0.85      0.79      0.82       367
           1       0.12      0.16      0.14        62

    accuracy                           0.70       429
   macro avg       0.48      0.48      0.48       429
weighted avg       0.74      0.70      0.72       429



# Saving Model

In [24]:
model.save('trained_model.keras')